<a href="https://colab.research.google.com/github/zoya-ivanova/BigData/blob/main/Spark_Apache3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spark Apache

### Инструменты работы и визуализации ч.2

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=22f47ae08bd34c28ee897c8b2ceea26d0e7dcda6b96d8b31567b6f6b88bc8603
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

In [7]:
spark = SparkSession.builder.appName("WindowFunctionExample").getOrCreate()

# Создаем DF
data = [("2023-11-20", "Electronics", 100, 12000), ("2023-11-21", "Electronics", 110, 13000), ("2023-11-22", "Electronics", 105, 12500),
  ("2023-11-20", "Clothing", 300, 15000), ("2023-11-21", "Clothing", 280, 14000), ("2023-11-22", "Clothing", 320, 16000),
  ("2023-11-20", "Books", 150, 9000), ("2023-11-21", "Books", 200, 12000), ("2023-11-22", "Books", 180, 10000)]

df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])

df.show()


+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [10]:
# Вычисляем среднюю выручку для каждой категории продуктов по дням
df_avg_revenue = df.groupBy("category", "date").agg(avg("revenue").alias("avg_revenue"))

# Выполняем операцию pivot, чтобы получить таблицу с категориями в строках и днями в столбцах
df_pivot = df_avg_revenue.groupBy("category").pivot("date").agg(avg("avg_revenue"))

# Выводим результирующий DataFrame
df_pivot.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [11]:
# Останавливаем SparkSession
spark.stop()

Spark Apache (семинары)
Инструменты работы и визуализации ч.2

In [4]:
spark = SparkSession.builder.appName("WindowFunctionExample").getOrCreate()

# Создаем DF
data = [("A", 1), ("A", 2), ("A", 3), ("B", 4), ("B", 5), ("B", 6)]
df = spark.createDataFrame(data, ["Group", "Value"])

# Определяем окно для каждой группы
windowSpec = Window.partitionBy("Group").orderBy("Value")

# Вычисление скользящего среднего
df = df.withColumn("MovingAverage", avg("Value").over(windowSpec.rowsBetween(-2, 0)))

df.show()

+-----+-----+-------------+
|Group|Value|MovingAverage|
+-----+-----+-------------+
|    A|    1|          1.0|
|    A|    2|          1.5|
|    A|    3|          2.0|
|    B|    4|          4.0|
|    B|    5|          4.5|
|    B|    6|          5.0|
+-----+-----+-------------+



### Типы оконных функций:

1/ Ранжирующие
row_number
rank
percent_rank
dense_rank
ntile

2/ Аналитические
lag
lead
cume_dist
nth_value

3/ Аггрегирующие
min/max
count
std

### Параметры:

1.PartitionBy()

2.OrderBy()

3.rowsBetweens(start, end)

4.rangeBetwen(start, end)

Window.unbdoudedPreceding, Window.unboudedFolliwing, Window.currentRow


### Окна нужны:

1 Гибкость в анализе данных

2 Сохранение идентичности исх данных

3 Поддержка сложных аналит запросов

4 Оптимизация производительности

5 Поддержка различных типов окон


In [ ]:
from pyspark.sql.window import Window

# Определение окна с разделением на партиции по столбцу "country" и ортировкой по столбцу "data"
window = Window.partitionBy("country").orderBy("data")

# Определение окна с границами, охватывающими строки от начала до текущей строки
window = Window.orderBy("data").rowsBetween(Window.unbdoudedPreceding, Window.currentRow)

# Определение окна с границами, охватывающими строки от 3 до текущей строки
window = Window.orderBy("data").rowsBetween(-3, Window.currentRow)

# Определение окна с границами, охватывающими строки от 3 строк до 3 строк после текущей строки
window = Window.orderBy("data").rowsBetween(-3, -3)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

# Определение окна с разделением на партиции по столбцу "departament" и ортировкой по столбцу "salary"
windowSpec = Window.partitionBy("departament").orderBy("salary")

# Применение оконной функции avg с использованием rowsBetween
df.withColumn("avg_salary", avg("salary").over(windowSpec.rowsBetween(Window.unbdoudedPreceding, Window.currentRow)))

# Применение оконной функции avg с использованием rangeBetween
df.withColumn("avg_salary_range", avg("salary").over(windowSpec.rowsBetween(Window.unbdoudedPreceding, 1000)))

In [ ]:
rowsBetween(Window.correntRow, 1)  # есть некое окно, включающее одну строки и одну после нее
rangeBetween(Window.correntRow, 1)  # есть некое окно, включающее строки значения при этом в столбце которого в пределах 1 от тек значения

In [ ]:
from pyspark.sql import Window
from pyspark.sql import functions as func

# Определение окна с использованием rowsBetween
window1 = Window.partitionBy("category").orderBy("id").rowsBetween(Window.correntRow, 1)

# Определение окна с использованием rangeBetween
window2 = Window.partitionBy("category").orderBy("id").rangeBetween(Window.correntRow, 1)

# Применение оконной функции DF
df.withColumn("sum_rows", func.sum("id").over(window1)) \
  .withColumn("sum_range", func.sum("id").over(window2)) \
  .show()

In [ ]:
from pyspark.sql import functions as udf
from pyspark.sql.types import IntegerType

# Определение пользовательской функции
def add_one(x):
  return x + 1

# Преобразование функции в UDF
add_one_udf = udf(add_one, IntegerType())

# Использование UDF в DF
df = spark.createDataFrame([(1,), (2,), (3,)], ['value'])
df.withColumn("value_plus_one", add_one_udf(df["value"])).show()

входные/выходные данные для UDF

1/IntergerType

2/DoubleType

3/StringType

4/BooleanType

5/ArrayType

6/MapType

7/StructType